# Section

In [ ]:
from pint import UnitRegistry
import sympy
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import sys
%matplotlib inline
from IPython.display import display

In [ ]:
sys.path.append('./')
from Section import Section

initialization

In [ ]:
ureg = UnitRegistry()
sympy.init_printing()

Define sympy parameters

In [ ]:
A, A0, t, t0, a, b, h, L = sympy.symbols('A A_0 t t_0 a b h L')

define graph

In [ ]:
stringers = {1:[(sympy.Integer(0),h),A],
             2:[(a/2,h),A0],
             3:[(a,h),A],
             4:[(a-b,sympy.Integer(0)),A],
             5:[(b,sympy.Integer(0)),A]}

panels = {(1,2):t,
          (2,3):t,
          (3,4):t,
          (4,5):t,
          (5,1):t}

In [ ]:
S1 = Section(stringers, panels)

In [ ]:
S1.symmetry

In [ ]:
values = [(A, 150 * ureg.millimeter**2),(A0, 250  * ureg.millimeter**2),(a, 80 * ureg.millimeter), \
          (b, 20 * ureg.millimeter),(h, 75 * ureg.millimeter),(L, 2000 * ureg.millimeter)]
datav = [(v[0],v[1].magnitude) for v in values]

In [ ]:
start_pos={ii: [S1.g.node[ii]['ip'][i].subs(datav) for i in range(2)] for ii in S1.g.nodes() }

In [ ]:
plt.figure(figsize=(12,8),dpi=300)
nx.draw(S1.g,with_labels=True, pos=start_pos)
plt.arrow(0,0,20,0)
plt.arrow(0,0,0,20)
#plt.text(0,0, 'CG', fontsize=24)
plt.axis('equal')
plt.title("Section in starting reference Frame",fontsize=16);

In [ ]:
positions={ii: S1.g.node[ii]['pos'].subs(datav) for ii in S1.g.nodes() }

In [ ]:
plt.figure(figsize=(12,8),dpi=300)
nx.draw(S1.g,with_labels=True, pos=positions)
plt.plot([0],[0],'o',ms=12)
plt.text(0,0, 'CG', fontsize=24)
plt.axis('equal')
plt.title("Section in pricipal reference Frame",fontsize=16);

## Compute axial loads in Stringers

We first define some symbols:

In [ ]:
Tx, Ty, Nz, Mx, My, Mz, F, ry, ry, mz = sympy.symbols('T_x T_y N_z M_x M_y M_z F r_y r_x m_z')

In [ ]:
S1.set_loads(Tx, Ty, Nz, Mx, My, Mz)

In [ ]:
S1.compute_stringer_actions()

In [ ]:
S1.N